In [2]:
:dep candle-core = "0.8.1"
:dep candle-nn = "0.8.1"

In [3]:
use candle_core::{DType, Device, Module, ModuleT, NdArray, Tensor, Var, D};
use candle_nn::linear::{Linear, linear};
use candle_nn::loss::mse;
use candle_nn::var_builder::{VarBuilder, SimpleBackend};
use candle_nn::{Optimizer, VarMap, SGD};

In [4]:
let device = Device::Cpu;
let weight = 0.7;
let bias = 0.3;

let start = 0.0;
let end = 1.0;
let step = 0.02;
let X = Tensor::arange_step(start, end, step, &device)?;
// let mut y = (weight * X.clone())?;
// y = (y + bias)?;
let y = ((weight * X.clone())? + bias)?;
let x_vec = X.clone().to_vec1::<f64>()?;
let y_vec = y.clone().to_vec1::<f64>()?;
println!("{:?}", &x_vec[..10]);
println!("{:?}", &y_vec[..10]);

let train_split: usize = (0.8 * x_vec.len() as f32) as usize;
println!("Train: {}, Test:{}\n", train_split, x_vec.len() - train_split as usize);

let X_train = Tensor::from_slice(&x_vec[..train_split], (train_split,1), &device)?;
let y_train = Tensor::from_slice(&y_vec[..train_split], (train_split, 1), &device)?;
let x_test = Tensor::from_slice(&x_vec[train_split..x_vec.len()], (x_vec.len() - train_split, 1), &device)?;
let y_test = Tensor::from_slice(&y_vec[train_split..y_vec.len()], (y_vec.len() - train_split, 1), &device)?;
println!("X_train: {}", X_train);
println!("Y_train: {}", y_train);
println!("x_test: {}", x_test);
println!("y_test: {}", y_test);

let varmap = VarMap::new();
let vb = VarBuilder::from_varmap(&varmap, DType::F64, &Device::Cpu);

let model: Linear = linear(1, 1, vb)?;
println!("Weight: {}", model.weight());
match model.bias() {
    Some(b) => println!("Bias: {}", b),
    None => {}
};

let y_preds = model.forward(&x_test)?;
println!("{}", y_preds);

let w = Var::from_tensor(&model.weight())?;
let b = Var::from_tensor(&model.bias().expect(""))?;
let mut opt = SGD::new(vec![w.clone(), b.clone()], 0.004)?;

for epoch in 1..10 {
    //1.  Forward pass on the training data
    let y_preds = model.forward(&X_train)?;

    //2.  Calculate the loss, for this we will use Mean Squared Error (mse) from candle
    let loss = mse(&y_preds, &y_train)?;
    println!("Loss: {}", loss);

    //3.  Candle does not have a zero_grad mechanism
    //4.  Get a GradStore from the loss
    let bp = loss.backward()?;
    //5. progress the optimzer
    opt.step(&bp);

    println!("Weight: {}", model.weight());
    match model.bias() {
        Some(b) => println!("Bias: {}", b),
        None => {}
    };

}


[0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12000000000000001, 0.14, 0.16, 0.18]
[0.3, 0.314, 0.32799999999999996, 0.34199999999999997, 0.356, 0.37, 0.384, 0.398, 0.412, 0.426]
Train: 40, Test:10

X_train: [[0.0000],
 [0.0200],
 [0.0400],
 [0.0600],
 [0.0800],
 [0.1000],
 [0.1200],
 [0.1400],
 [0.1600],
 [0.1800],
 [0.2000],
 [0.2200],
 [0.2400],
 [0.2600],
 [0.2800],
 [0.3000],
 [0.3200],
 [0.3400],
 [0.3600],
 [0.3800],
 [0.4000],
 [0.4200],
 [0.4400],
 [0.4600],
 [0.4800],
 [0.5000],
 [0.5200],
 [0.5400],
 [0.5600],
 [0.5800],
 [0.6000],
 [0.6200],
 [0.6400],
 [0.6600],
 [0.6800],
 [0.7000],
 [0.7200],
 [0.7400],
 [0.7600],
 [0.7800]]
Tensor[[40, 1], f64]
Y_train: [[0.3000],
 [0.3140],
 [0.3280],
 [0.3420],
 [0.3560],
 [0.3700],
 [0.3840],
 [0.3980],
 [0.4120],
 [0.4260],
 [0.4400],
 [0.4540],
 [0.4680],
 [0.4820],
 [0.4960],
 [0.5100],
 [0.5240],
 [0.5380],
 [0.5520],
 [0.5660],
 [0.5800],
 [0.5940],
 [0.6080],
 [0.6220],
 [0.6360],
 [0.6500],
 [0.6640],
 [0.6780],
 [0.6920],
 [0.7060],
 

()

Bias: [-0.2054]
Tensor[[1], f64]
Loss: [3.6219]
Tensor[[], f64]
Weight: [[-2.4943]]
Tensor[[1, 1], f64]
Bias: [-0.1913]
Tensor[[1], f64]
Loss: [3.5613]
Tensor[[], f64]
Weight: [[-2.4875]]
Tensor[[1, 1], f64]
Bias: [-0.1774]
Tensor[[1], f64]
Loss: [3.5018]
Tensor[[], f64]
Weight: [[-2.4808]]
Tensor[[1, 1], f64]
Bias: [-0.1637]
Tensor[[1], f64]
Loss: [3.4434]
Tensor[[], f64]
Weight: [[-2.4741]]
Tensor[[1, 1], f64]
Bias: [-0.1500]
Tensor[[1], f64]
Loss: [3.3861]
Tensor[[], f64]
Weight: [[-2.4675]]
Tensor[[1, 1], f64]
Bias: [-0.1365]
Tensor[[1], f64]
Loss: [3.3298]
Tensor[[], f64]
Weight: [[-2.4609]]
Tensor[[1, 1], f64]
Bias: [-0.1232]
Tensor[[1], f64]
Loss: [3.2745]
Tensor[[], f64]
Weight: [[-2.4544]]
Tensor[[1, 1], f64]
Bias: [-0.1099]
Tensor[[1], f64]
